In [1]:
import torch
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from utils import *

[2023-11-08 10:53:19,123] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [9]:
base_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-chat-hf',
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,   
)
base_model.model_parellal = True

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
model = PeftModel.from_pretrained(base_model, 'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora')
model = model.eval()

In [5]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')
tokenizer.padding_side = "right"
tokenizer.pad_token_id = tokenizer.eos_token_id

In [6]:
test_dataset = load_from_disk('data/fingpt-forecaster-dow30v3-20221231-20230531-llama/')['test']

In [7]:
def test_demo(model, tokenizer, prompt):

    inputs = tokenizer(
        prompt, return_tensors='pt',
        padding=False, max_length=4096
    )
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
        
    res = model.generate(
        **inputs, max_length=4096, do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True
    )
    output = tokenizer.decode(res[0], skip_special_tokens=True)
    return output    
    # return res

In [8]:
answers, gts = [], []

for i in range(len(test_dataset)):
    prompt = test_dataset[i]['prompt']
    output = test_demo(model, tokenizer, prompt)
    answer = re.sub(r'.*\[/INST\]\s*', '', output, flags=re.DOTALL)
    gt = test_dataset[i]['answer']
    print('\n------- Prompt ------\n')
    print(prompt)
    print('\n------- LLaMA2 Finetuned ------\n')
    print(answer)
    print('\n------- GPT4 Groundtruth ------\n')
    print(gt)
    print('\n===============\n')
    answers.append(answer)
    gts.append(gt)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



------- Prompt ------

[INST]<<SYS>>
You are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. Your answer format should be as follows:

[Positive Developments]:
1. ...

[Potential Concerns]:
1. ...

[Prediction & Analysis]
Prediction: ...
Analysis: ...

<</SYS>>

[Company Introduction]:

American Express Co is a leading entity in the Financial Services sector. Incorporated and publicly traded since 1977-05-18, the company has established its reputation as one of the key players in the market. As of today, American Express Co has a market capitalization of 106006.97 in USD, with 736.46 shares outstanding.

American Express Co operates primarily in the US, trading under the ticker AXP on the NEW YORK STOCK EXCHANGE, INC.. As a dominant force in the Financ

In [10]:
gts[0]

"[Positive Developments]:\n1. The declaration of a regular quarterly dividend, amounting to $0.60 per common share, indicates a steady cash return to shareholders. This can boost investor confidence, leading to increased demand for AXP shares.\n2. A company executive's participation in major industry conferences offers the chance to elaborate on American Express's growth strategy and financial performance. Such events could provide positive exposure for the company, potentially driving stock price gains.\n3. The article highlighting American Express's double-digit revenue growth and aggressive share buyback strategy illustrates strong company performance, a factor that can lead to increased investor interest.\n\n[Potential Concerns]:\n1. The interest rate hike by the Federal Reserve could put pressure on financial institutions, affecting their loans' profitability and potentially leading to a decline in stock prices.\n2. According to one report, concerns exist that credit card profitab

In [11]:
answers[0]

"[Positive Developments]:\n1. The company's Vice Chairman, Douglas E. Buckminster, will participate in the SVB MoffettNathanson Technology, Media, and Telecom Conference, which could potentially increase investor confidence and interest in the company.\n2. American Express declared a regular quarterly dividend, which may attract dividend-focused investors.\n3. The company's stock price increased from 146.82 to 151.80 in the week from 2023-05-14 to 2023-05-21, indicating potential upward trends.\n\n[Potential Concerns]:\n1. The company's stock price has been declining since the beginning of May, which may indicate a potential downward trend.\n2. There have been concerns about the company's profitability after a strong 2022, which could indicate a potential peak in profitability.\n3. The company's stock has been mentioned in articles discussing the potential risks of fintech companies and the regional banking crisis, which could impact the company's profitability and stock price.\n\n[Pre

In [12]:
calc_metrics(answers, gts)


Binary Accuracy: 0.61  |  Mean Square Error: 5.93

Rouge Score of Positive Developments: {'rouge1': 0.43196333904653406, 'rouge2': 0.15722817931423375, 'rougeL': 0.264313106777193}

Rouge Score of Potential Concerns: {'rouge1': 0.39554620999107204, 'rouge2': 0.12632646861001814, 'rougeL': 0.24735208697254654}

Rouge Score of Summary Analysis: {'rouge1': 0.42928213837028545, 'rouge2': 0.1354372841499895, 'rougeL': 0.2320299101133297}


{'valid_count': 57,
 'bin_acc': 0.6140350877192983,
 'mse': 5.9298245614035086,
 'pros_rouge_scores': {'rouge1': 0.43196333904653406,
  'rouge2': 0.15722817931423375,
  'rougeL': 0.264313106777193},
 'cons_rouge_scores': {'rouge1': 0.39554620999107204,
  'rouge2': 0.12632646861001814,
  'rougeL': 0.24735208697254654},
 'anal_rouge_scores': {'rouge1': 0.42928213837028545,
  'rouge2': 0.1354372841499895,
  'rougeL': 0.2320299101133297}}